# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"¶

### Задание 1.
Обязательная часть  
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.  

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:  

KEYWORDS = ['python', 'парсинг']  

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).  

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>  

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import json

In [170]:
KEYWORDS = ['python', 'парсинг', 'sony', 'обратная', 'водител', 'карьер', 'интернет']

In [148]:
res = requests.get('https://habr.com/ru/all/') 
soup = BeautifulSoup(res.text, 'html.parser')
posts = soup.find_all('article', class_='post')

In [171]:
habrframe = pd.DataFrame() # создаём пустую таблицу
for post in posts: # цикл для извлечения параметров постов
    cnt = 0 # счетчик проверки вхождения KEYWORDS
    text = post.find('div', class_ = 'post__text').text.lower()
    # проверка вхождения заданных слов в заголовок
    for words in KEYWORDS:
        if words in text:
            cnt +=1 
    if cnt > 0:
        heads = post.find('h2', class_ = 'post__title').text #извлекаем заголовкистатей
        time = post.find('span', class_ = 'post__time').text # извлекаем время
        link = post.find('h2', class_ = 'post__title').find('a').get('href') # извлекаем ссылку
        row = {'time': time, 'heads': heads, 'links': link} # создаем словарь, для дальнейшего создания строки
        habrframe = pd.concat([habrframe, pd.DataFrame([row])])
habrframe

,time,heads,links
0,сегодня в 12:23,\nНовый продукт как результат карьерных измене...,https://habr.com/ru/post/530472/
0,сегодня в 11:28,\nСистемы отслеживания состояния водителей\n,https://habr.com/ru/company/itelma/blog/532640/
0,сегодня в 01:09,\nПора ли отказываться от обычного Интернета в...,https://habr.com/ru/post/532644/


### Задание 2.
Обязательная часть  
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:  
EMAIL = [xxx@x.ru, yyy@y.com]  

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>  

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.  

In [2]:
EMAIL = ['listok@mail.ru', 'sobaka@mail.ru', 'koshka@mail.ru']

In [9]:
url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
mail = {'emailAddresses': ['listok@mail.ru']}
headers = {'Accept': 'application/json, text/plain, */*',
           'Accept-Encoding': 'gzip, deflate, br',
           'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
           'Connection': 'keep-alive',
           'Content-Length': '41',
           'Content-Type': 'application/json;charset=UTF-8',
           'Host': 'identityprotection.avast.com',
           'Origin': 'https://www.avast.com',
           'Referer': 'https://www.avast.com/',
           'Sec-Fetch-Dest': 'empty',
           'Sec-Fetch-Mode': 'cors',
           'Sec-Fetch-Site': 'same-site',
           'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36',
           'Vaar-Header-App-Product': 'hackcheck-web-avast',
           'Vaar-Version': '0'
          }

In [17]:
mailframe = pd.DataFrame() # создаем пустой дата фрейм для дальнейшего наполнения
for email in EMAIL: # итерирование по списку электронной почты
    avastdata = requests.post(url, json ={'emailAddresses': [email]} , headers = headers) # post запрос на сервер
    print(avastdata) # вывод статуса для контроля прохождения цикла
    if avastdata.text == 'null': # проверка на корректность ответа от сервера
        print("c сервера вернулся пустой ответ")
        continue
    time.sleep(0.9) 
    maildata = json.loads(avastdata.text)['breaches'] # преобразование ответа от сервера в словарь
    for rowvalue in maildata.values(): #итерация по словарю, запись значений в строку
        datemail = rowvalue['publishDate']
        sourcemail = rowvalue['site']
        descriptionmail = rowvalue['description']
        row = {'Email': email, 'date': datemail, 'source': sourcemail, 'description': descriptionmail}
        mailframe = pd.concat([mailframe, pd.DataFrame([row])])

<Response [200]>
<Response [200]>
<Response [200]>


In [18]:
mailframe

,Email,date,source,description
0,listok@mail.ru,2019-03-28T00:00:00Z,verifications.io,Big data e-mail verification platform verifica...
0,listok@mail.ru,2020-05-21T00:00:00Z,vk.com,"At some time in 2020, the Russian social netwo..."
0,listok@mail.ru,2017-08-30T00:00:00Z,sprashivai.ru,"In January 2015, Sprashivai.ru's user database..."
0,listok@mail.ru,2016-11-01T00:00:00Z,qip.ru,"In 2011, Russian instant messaging service pro..."
0,listok@mail.ru,2019-11-21T00:00:00Z,faucethub.io,"At an unconfirmed date, cryptocurrency micropa..."
...,...,...,...,...
0,koshka@mail.ru,2020-12-10T00:00:00Z,geoflor.ru,"In November 2020, a collection of over 23,000 ..."
0,koshka@mail.ru,2020-04-09T00:00:00Z,vkmix.com,"At an unconfirmed date, the Russian social med..."
0,koshka@mail.ru,2020-11-05T00:00:00Z,vimeworld.ru,"In January 2018, the Russian Minecraft server ..."
0,koshka@mail.ru,2020-12-03T00:00:00Z,forumy.ru,"In November 2020, a collection of over 23,000 ..."
